# Memory

In [30]:
import { ChatMessageHistory } from 'langchain/stores/message/in_memory'
import { HumanMessage, AIMessage } from '@langchain/core/messages'

const history = new ChatMessageHistory()

await history.addMessage(new HumanMessage('Hello!'))
await history.addMessage(new AIMessage('Hi!'))

const messages = await history.getMessages()
console.log(messages)

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "Hello!", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello!",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "Hi!", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi!",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [31]:
// import env config

import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [32]:
import { ChatPromptTemplate, MessagesPlaceholder } from '@langchain/core/prompts'

import { ChatOpenAI } from '@langchain/openai'

const chatModel = new ChatOpenAI()

const prompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.
        You are talkative and provides lots of specific details from its context. 
        If the you does not know the answer to a question, it truthfully says you do not know.`],
    new MessagesPlaceholder('history_message')
])

const chain = prompt.pipe(chatModel)

In [33]:
import { ChatMessageHistory } from 'langchain/stores/message/in_memory'
import { HumanMessage } from '@langchain/core/messages'

const history = new ChatMessageHistory()
await history.addMessage(new HumanMessage('Hello! My name is Bob.'));

const res = await chain.invoke({
    history_message: await history.getMessages()
})

console.log(res)

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello Bob! How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello Bob! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 10, promptTokens: 72, totalTokens: 82 },
    finish_reason: "stop"
  }
}


In [34]:
await history.addMessage(res)
await history.addMessage(new HumanMessage('What is my name?'))

const res2 = await chain.invoke({
    history_message: await history.getMessages()
})

console.log(res2)

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Your name is Bob.",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Your name is Bob.",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 5, promptTokens: 95, totalTokens: 100 },
    finish_reason: "stop"
  }
}


## Manully manage memory

In [67]:
import { ChatOpenAI } from '@langchain/openai'
import { ChatPromptTemplate, MessagesPlaceholder } from '@langchain/core/prompts'
import { ChatMessageHistory } from 'langchain/stores/message/in_memory'
import { RunnableWithMessageHistory } from '@langchain/core/runnables'


const chatModel = new ChatOpenAI()
const prompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.
        You are talkative and provides lots of specific details from its context. 
        If the you does not know the answer to a question, it truthfully says you do not know.`],
    new MessagesPlaceholder('history_message'),
    ['human', '{input}']
])

const history = new ChatMessageHistory()
const chain = prompt.pipe(chatModel)

const chainWithHistory = new RunnableWithMessageHistory({
    runnable: chain,
    getMessageHistory: _sessionId => history,
    inputMessageKey: 'input',
    historyMessagesKey: 'history_message' // spelling issue 's' missing
})


In [69]:
const res1 = await chainWithHistory.invoke({
    input: 'Hello! My name is Bob.',
},{
    configurable: { sessionId: "none" }
})

console.log(res1)

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello, Bob! How can I assist you today?",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello, Bob! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 11, promptTokens: 104, totalTokens: 115 },
    finish_reason: "stop"
  }
}


In [70]:
const res2 = await chainWithHistory.invoke({
    input: 'What is my name?',
},{
    configurable: { sessionId: "none" }
})

console.log(res2)

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Your name is Bob.",
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Your name is Bob.",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 5, promptTokens: 128, totalTokens: 133 },
    finish_reason: "stop"
  }
}


In [71]:
await history.getMessages()

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "hi, my name is Kai",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi, my name is Kai",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hello, Kai! It's nice to meet you. How can I assist you today?",
      additional_kwargs: { function_call: undefined, tool_calls: undefined },
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello, Kai! It's nice to meet you. How can I assist you today?",
    name: undefined,
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {
      tokenUsage: { completionTokens: 18, promptTokens: 71, totalTokens: 89 },
      finish_reason: "stop"
    }
  },
  HumanMessage {
    lc_serializable: true,
   

## Auto Chat history abstraction